In [1]:

%matplotlib widget
# Load packages and settings
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns

import geopandas as gpd

import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)

from matplotlib import cm # Colormaps
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# ax1.spines['top'].set_visible(False) 

import os
# import csv
import math

from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

# Define running mean functions
def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]

saveFigures is set to: True
Done loading packages


In [2]:
# Define paths
rootdir_data = os.getcwd() +"\\..\\DanskeData\\" 

path_data = rootdir_data + "ssi_data\\"
path_dash = rootdir_data + "ssi_dashboard\\"
path_vacc = rootdir_data + "ssi_vacc\\"
path_figs = os.getcwd() +"\\..\\Figures\\" 


path_figs = path_figs + "Maps\\KommuneAnimation"

In [3]:
gdf = gpd.read_file("C:/Users/Rasmus/Desktop/Kommuner/Kommune.shp")

# Only use most recent mapdata
gdf = gdf[gdf.til == np.max(gdf.til.unique())]

In [4]:
# Walk to relavant folder
latestsubdir = list(os.walk(path_dash))[0][1][-1]
latestdir = path_dash + latestsubdir
df = pd.read_csv(latestdir+'/Kommunalt_DB/07_bekraeftede_tilfaelde_pr_dag_pr_kommune.csv',encoding='latin1',delimiter = ';')

df['Dato'] = pd.to_datetime(df['Dato'])

In [5]:

def getKommuneCount(curKommune):
    kommune_df = df.loc[df["Kommunenavn"] == curKommune]
    
    # Cut out last data point
    kommune_df = kommune_df.iloc[:-1,:]
    # firstDate = np.datetime64(kommune_df.loc[kommune_df.index[0],'Dato'])-np.timedelta64(1,'D')
    # firstDate = np.datetime64('2021-11-01')
    # lastDate = np.datetime64(kommune_df.loc[kommune_df.index[-1],'Dato'])
    # Find number of citizens in region
    latestsubdir = list(os.walk(path_dash))[0][1][-1]
    latestdir = path_dash + latestsubdir
    df_kommunekort = pd.read_csv(latestdir+'/Kommunalt_DB/10_Kommune_kort.csv',encoding='latin1',
                                 delimiter = ';')
    df_kommunekort = df_kommunekort.set_index("Kommunenavn")

    # kommune_nr = kommune_df.Kommune.iloc[0]
    # kommune_df['Procent andel smittede'] = (kommune_df['Bekræftede tilfælde']/antal_borgere(curKommune))*100

    curDays = kommune_df['Dato'].values
    antal_borgere = df_kommunekort["Antal borgere"][curKommune]
    curPerc = ((kommune_df['Bekræftede tilfælde']/antal_borgere)*100).values
    curCount = kommune_df['Bekræftede tilfælde'].values 
    
    indexToUse = curDays <= (np.datetime64(latestsubdir[-10:])-np.timedelta64(2,'D'))
    curCount = curCount[indexToUse]
    curPerc = curPerc[indexToUse]
    curDays = curDays[indexToUse]

    return curDays,curCount,curPerc


In [6]:
# Calculate growthmeasure
# kommunenavn = 'København'
# kommunenavne = ["København","Århus","Aalborg","Odense","Roskilde","Ishøj","Frederiksberg","Hvidovre","Greve","Rødovre","Skanderborg"]
# kommunenavne = ["København","Århus","Aalborg","Odense","Roskilde","Ishøj","Frederiksberg"]
kommunenavne = df.Kommunenavn.unique()

df_meas = pd.DataFrame()

curDays,curCount,curPerc = getKommuneCount('København')
# df_meas['Date'] = [curDays[-1]]
dateRange = np.arange(np.datetime64('2021-12-01'),np.datetime64('today'))
df_meas['Date'] = dateRange[14:]

# fig,ax1 = plt.subplots()
for kommunenavn in kommunenavne:
# for kommunenavn in kommunenavne[:4]:
    curDays,curCount,curPerc = getKommuneCount(kommunenavn)

    firstDate = np.datetime64('2021-12-01')
    curIndex = (curDays >= firstDate)
    curDays = curDays[curIndex]
    curCount = curCount[curIndex]
    curPerc = curPerc[curIndex]

    
    posIndices = [np.where(x == np.arange(np.datetime64('2021-12-01'),np.datetime64('today')))[0][0] for x in curDays]

    try:
        curDays = curDays[posIndices]
        curCount = curCount[posIndices]
        curPerc = curPerc[posIndices]

        simplerCount = curCount[7:] - curCount[:-7]
        simplerCount2 = (curCount[7:] - curCount[:-7])/curCount[:-7]
        curMeasure1 = rnMean(simplerCount2,7)
        curMeasure2 = rnMean(np.sign(simplerCount2),7)
        # curMeasure2 = rnMean(np.sign(simplerCount2),4)

        # # ax1.plot(curDays[13:],curMeasure2,'*:',label=kommunenavn)

        # # df_meas['Dates'] = curDays[13:]
        # # df_meas[kommunenavn] = curMeasure2

        # # df_meas[kommunenavn] = [curMeasure2[-1]]

        # df_meas['Date'] = curDays[13:]
        df_meas[kommunenavn] = curMeasure2
    except:
        2+2
        # df_meas[kommunenavn] = np.zeros(dateRange.shape)


# df_meas
# ax1.set_xlim(left=np.datetime64('2021-11-01'))

# ax1.legend()
# ax1.grid()
# ax1.set_ylim([-1.5,1.5])

# df_meas

In [7]:
np.where(df_meas.Date == np.datetime64('2022-01-31'))[0][0]

47

In [8]:
allKoms = df_meas.columns[1:]
# [x.split(' Kommun')[0] for x in gdf.navn]

curCount = 0
for curDate in df_meas.Date.values:
    fig,ax1 = plt.subplots() 
    # gdf.plot(color='gray',ax=ax1)


    # curDate = np.datetime64('today')
    # curDate = np.datetime64('2022-01-31')

    # for x, y, label in zip(gdf.geometry.centroid.x, gdf.geometry.centroid.y, gdf.navn):
    #     ax1.annotate(label,xy=(x,y),xytext=(3,3),textcoords="offset points")

    import matplotlib
    cmap = matplotlib.cm.get_cmap('seismic')

    for curKom in allKoms:
        # curKom = 'Horsens'

            
        # curVal = df_meas[curKom].values[0]
        # curVal = (1+df_meas[curKom].values[0])/2

        curIndex = np.where(df_meas.Date == (curDate))[0][0]
        # curIndex = np.where(df_meas.Date == (curDate-np.timedelta64(1,'D')))[0][0]
        curVal = (1+df_meas[curKom].values[curIndex])/2
        
        
        curColor = cmap(curVal)
        
        if (curKom == 'Aabenraa'):
            curKom = 'Åbenrå'
        if (curKom == 'Nordfyn'):
            curKom = 'Nordfyns'
        if (curKom == 'København'):
            curKom = 'Københavns'
        if (curKom == 'Bornholm'):
            curKom = 'Bornholms'
        if (curKom == 'Faaborg-Midtfyn'):
            curKom = 'Fåborg-Midtfyn'
        if (curKom == 'Lyngby-Taarbæk'):
            curKom = 'Lyngby-Tårbæk'
            

        curgdf = gdf[gdf.navn == (curKom+' Kommune')]
        curgdf.plot(ax=ax1,color=curColor)

    # Hand-crafted legend
    import matplotlib.patches as patches
    # posVals = np.arange(-7,8,2)
    posVals = np.arange(-1,1.1,2/7)
    posValsStr = [str(int(np.round(7*x))) for x in posVals]
    posValsStr = [
        '7 dages vækst',
        '6 dages vækst, 1 dags fald',
        '5 dages vækst, 2 dages fald',
        '4 dages vækst, 3 dages fald',
        '3 dages vækst, 4 dages fald',
        '2 dages vækst, 5 dages fald',
        '1 dags vækst, 6 dages fald',
        '7 dages fald',
    ]

    for i in range(len(posVals)):
        curVal = posVals[i]
        curVal = 1-(1+posVals[i])/2
        curColor = cmap(curVal)
        curStr = posValsStr[i]
        x0 = 680000
        y0 = 6.35e6
        wi = 9000
        he = 10000
        cury = y0-he*i -he*0.5*i
        rect = patches.Rectangle((x0,cury),wi,he,linewidth=2,edgecolor='k',facecolor=curColor)
        ax1.add_patch(rect)
        # ax1.annotate('asdf',xy=(x0,y0))
        ax1.text(x0+wi*2,cury,curStr,ha='left')

    # Remove axes
    ax1.set_xticks([])
    ax1.set_yticks([])
    ax1.spines['right'].set_visible(False)
    ax1.spines['top'].set_visible(False)
    ax1.spines['left'].set_visible(False)
    ax1.spines['bottom'].set_visible(False)

    fileDate = pd.to_datetime(curDate+np.timedelta64(1,'D')).strftime('%Y-%m-%d')
    titleDate = pd.to_datetime(curDate+np.timedelta64(1,'D')).strftime('%d. %B')
    # fileDate = pd.to_datetime(curDate).strftime('%Y-%m-%d')
    # titleDate = pd.to_datetime(curDate).strftime('%d. %B')
    ax1.set_title(titleDate) 

    if saveFigures:
        fig.savefig(path_figs+f'\\KommuneUdvikling_{fileDate}')
        fig.savefig(path_figs+f'\\frames\\frame{curCount}')
        curCount = curCount + 1

# ax1.set_xlim([690000,750000])
# ax1.set_ylim([6.16e6,6.195e6])


# for i in range(len(posVals)):
#     curVal = posVals[i]
#     curVal = 1-(1+posVals[i])/2
#     curColor = cmap(curVal)
#     curStr = posValsStr[i]
#     x0 = 735000
#     y0 = 6.18e6
#     wi = 9000/4
#     he = 10000/6
#     cury = y0-he*i -he*0.5*i
#     rect = patches.Rectangle((x0,cury),wi,he,linewidth=2,edgecolor='k',facecolor=curColor)
#     ax1.add_patch(rect)
#     # ax1.annotate('asdf',xy=(x0,y0))
#     ax1.text(x0+wi*2,cury,curStr,ha='left')

# if saveFigures:
#     fig.savefig(path_figs+'/Maps/KommuneUdvikling_Hovedstaden')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
# # df_meas

# # curDays,curCount,curPerc = getKommuneCount(kommunenavn)
# # curDays
# [np.where(x == np.arange(np.datetime64('2021-12-01'),np.datetime64('today')))[0][0] for x in curDays]

# posIndices
np.where(df_meas.Date == curDate)[0][0]

51

In [10]:
# # Test af forskellige vækstmål
# curDays,curCount,curPerc = getKommuneCount('København')

# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

# ax1.plot(curDays,curCount,'.:',linewidth=1,color='k',markersize=2)

# meanDays = rnTime(curDays,7)
# # meanDays = curDays[6:]
# meanCount = rnMean(curCount,7)
# ax1.plot(meanDays,meanCount,color='k')

# # ax2.plot(meanDays[1:],np.diff(meanCount),'*--')

# simplerCount = curCount[7:] - curCount[:-7]
# # ax2.plot(meanDays[1:],simplerCount/7)

# simplerCount2 = (curCount[7:] - curCount[:-7])/curCount[:-7]
# # ax2.plot(meanDays[1:],simplerCount2,'.')
# ax2.plot(rnTime(meanDays[1:],7),rnMean(simplerCount2,7))
# ax2.plot(rnTime(meanDays[1:],4),rnMean(simplerCount2,4))
# # ax2.plot(rnTime(meanDays[1:],2),rnMean(simplerCount2,2))


# # ax2.plot(meanDays[1:],np.sign(simplerCount),'.')

# periodBack = 7
# ax2.plot(meanDays[periodBack:],rnMean(np.sign(simplerCount),periodBack),'.-')
# periodBack = 4
# ax2.plot(meanDays[periodBack:],rnMean(np.sign(simplerCount),periodBack),'.-')
# periodBack = 2
# ax2.plot(meanDays[periodBack:],rnMean(np.sign(simplerCount),periodBack),'.-')

# ax1.set_ylim(bottom=0)
# ax1.set_xlim(left=np.datetime64('2021-11-01'))
# ax2.set_ylim([-1.1,1.1])
# ax2.grid()